<a href="https://colab.research.google.com/github/kram-12/tweet-sentiment-analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing kaggle library

In [ ]:
! pip install kaggle

Configuring the path of kaggle.json file

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing dataset

In [ ]:
#API to fetch dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 80% 65.0M/80.9M [00:00<00:00, 230MB/s]
100% 80.9M/80.9M [00:00<00:00, 229MB/s]


In [ ]:
#Extracting compressed dataset
from zipfile import ZipFile
dataset="/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("Dataset is extracted.")

Dataset is extracted.


Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [ ]:
# loading the data from csv file to pandas dataframe
twitter_data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [ ]:
# naming the columns and reading the dataset
column_names=['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names=column_names, encoding="ISO-8859-1")

In [ ]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
# checking the distribution of target column after converting
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 -> Negative Tweet

1 -> Positive Tweet

**Stemming**

Process of reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
# checking the stemmed content
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
# seperating the data and label
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

Splitting the data to training data and test data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


Converting textual data to numerical data

In [ ]:
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train, X_test)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

Training the Machine Learning Model

Logistic Regression (Classification)

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

Accuracy Score

In [ ]:
# accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy Score on the training data:',training_data_accuracy)

Accuracy Score on the training data: 0.79871953125


In [ ]:
# accuracy score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy Score on the test data:',test_data_accuracy)

Accuracy Score on the test data: 0.77668125


Model accuracy = 77.66%

**Saving the trained model**

In [ ]:
import pickle

In [ ]:
filename="trained_model.sav"
pickle.dump(model,open(filename,'wb'))

Using the saved model for future predictions

In [ ]:
# loding the saved model
loaded_model=pickle.load(open("/content/trained_model.sav","rb"))

Model Check

In [ ]:
X_new=X_test[200]
print(Y_test[200])

1


In [ ]:
prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

[1]
Positive Tweet


**Model Check using a new tweet**

Tweet 1: Had the best time today at the park! The weather was perfect, and the sunset was absolutely stunning. Can't wait to go back! 🌅 #blessed #naturelove

In [ ]:
tweet_data1 = {
    'target': None,  # We'll predict this
    'id': 9999999999,  # Assign a dummy ID
    'date': '2023-11-17',  # Assign a dummy date
    'flag': 'None',  # Assign a dummy flag
    'user': 'dummy_user',  # Assign a dummy user
    'text': "Had the best time today at the park! The weather was perfect, and the sunset was absolutely stunning. Can't wait to go back! 🌅 #blessed #naturelove"
}

# Create a pandas DataFrame
dummy_tweet1 = pd.DataFrame([tweet_data1])

# Apply stemming function
dummy_tweet1['stemmed_content'] = dummy_tweet1['text'].apply(stemming)

# Vectorize using your existing vectorizer
dummy_tweet1_vectorized = vectorizer.transform(dummy_tweet1['stemmed_content'])

# Predict using your loaded model
prediction = loaded_model.predict(dummy_tweet1_vectorized)

# Assign the prediction to the 'target' column
dummy_tweet1['target'] = prediction[0]

# Print the prediction
if prediction[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

Positive Tweet


Tweet 2: Terrible customer service today. Waited over an hour just to get the wrong order. Definitely not coming back. 😡 #frustrated #neveragain

In [ ]:
tweet_data2 = {
    'target': None,  # We'll predict this
    'id': 9999999998,  # Assign a dummy ID
    'date': '2024-10-10',  # Assign a dummy date
    'flag': 'None',  # Assign a dummy flag
    'user': 'shyguy118',  # Assign a dummy user
    'text': "Terrible customer service today. Waited over an hour just to get the wrong order. Definitely not coming back. 😡 #frustrated #neveragain"
}

# Create a pandas DataFrame
dummy_tweet2 = pd.DataFrame([tweet_data2])

# Apply stemming function
dummy_tweet2['stemmed_content'] = dummy_tweet2['text'].apply(stemming)

# Vectorize using your existing vectorizer
dummy_tweet2_vectorized = vectorizer.transform(dummy_tweet2['stemmed_content'])

# Predict using your loaded model
prediction = loaded_model.predict(dummy_tweet2_vectorized)

# Assign the prediction to the 'target' column
dummy_tweet2['target'] = prediction[0]

# Print the prediction
if prediction[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

Negative Tweet
